<a href="https://colab.research.google.com/github/mrthomware/MakerSpace/blob/main/Intro_to_Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting your app to qdrant with Langchain

The first thing that you need to do is create an account on qdrant cloud and subsequently create a cluster. Qdrant cloud offers a free-forever 1GB cluster for your projects.

Once you get that, the the host and API keys on your account. Then you can follow these steps.

In [ ]:
!pip install langchain qdrant_client openai tiktoken

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
import qdrant_client
import os

In [ ]:
# create your client

os.environ['QDRANT_HOST'] =
os.environ['QDRANT_API_KEY'] =


client = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

In [ ]:
# create collection

os.environ['QDRANT_COLLECTION'] =

collection_config = qdrant_client.http.models.VectorParams(
        size=1536, # 768 for instructor-xl, 1536 for OpenAI
        distance=qdrant_client.http.models.Distance.COSINE
    )

client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    vectors_config=collection_config
)

True

In [ ]:
# create your vector store

os.environ['OPENAI_API_KEY'] =

embeddings = OpenAIEmbeddings()

vectorstore = Qdrant(
        client=client,
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        embeddings=embeddings
    )

In [ ]:
# add documents to your vector database

from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open("story.txt") as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

['72c466dfe701425b9cb83f10bb8f1c2d',
 '63f0aae63a0140a3acd064909c73ee1f',
 '32437187680f4eb7b2ec330a948ce60f',
 '99f9bb023a7e409fba4c714fa7b391b3',
 '167e58a62263493f98bb8837596b2017']

In [ ]:
# plug the vector store to your retrieval chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
    )

In [ ]:
query = "How many friends are there and what are their names?"
response = qa.run(query)

print(response)

 There are three friends and their names are Emily, Ben, and Sarah.
